In [49]:
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
import requests
import io

In [25]:
BASE_URL = "https://dijlovasok.hu/index.php/programgyujt"

In [26]:
def get_contents_of_page (url, headers = None) :
    req = requests.get(url, headers = headers if headers != None else {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
    })
    
    return req.content

In [29]:
def soupify (content) :
    return BeautifulSoup(content, "html.parser")

In [41]:
def get_links_from_main_page ():
    soup = soupify(get_contents_of_page(BASE_URL))
    # print(soup.prettify())
    program_hrefs = list(
        set(
            map(
                lambda href: href["href"], 
                filter(lambda href: href["href"].startswith("/index.php/programgyujt/"), 
                    soup.find_all("a", href = True)
                )
            )
        )
    )
    return program_hrefs    

In [58]:
def get_pdf_links_from_page (url):
    soup: BeautifulSoup = soupify(get_contents_of_page(url))
    pdf_hrefs = list(
            set(
                map(
                    lambda href: href["href"], 
                    filter(lambda href: href["href"].endswith(".pdf"), 
                        soup.find_all("a", href = True)
                    )
                )
            )
        )
    return pdf_hrefs

In [62]:
def get_pdf_texts () :

    pdf_texts = {}

    for page_url in get_links_from_main_page() :
        
        print(f"Scraping page {page_url}...")
        
        page_content = {}
        
        for pdf_url in get_pdf_links_from_page(BASE_URL + page_url) :
            
            print(f"Downloading pdf {pdf_url}...")
            
            pdf_content = []
            
            headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

            response = requests.get(url=pdf_url, headers=headers, timeout=120)
            on_fly_mem_obj = io.BytesIO(response.content)
            
            reader = PdfReader(on_fly_mem_obj)
            
            for page in reader.pages :
                pdf_content.append(page.extract_text())
                
            page_content[pdf_url] = pdf_content
            
        pdf_texts[page_url] = page_content
        
    return pdf_texts

In [63]:
pdf_texts = get_pdf_texts()

Scraping page /index.php/programgyujt/1457-kozep-osztalyu-feladatok-mb-program...
Scraping page /index.php/programgyujt/1031-elokeszito-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1217-vezetoszaras-feladat...
Scraping page /index.php/programgyujt/1035-kozep-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1008-fiatal-lovak-programjai...
Scraping page /index.php/programgyujt/1034-konnyu-kozep-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1037-nagydij-szintu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1033-konnyu-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1032-alapfoku-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1036-nehez-osztalyu-feladatok-hatalyos-2018-05-01...


In [64]:
pdf_texts

{'/index.php/programgyujt/1457-kozep-osztalyu-feladatok-mb-program': {'https://www.dijlovasok.hu/dokumentum/Programgyujtemeny_2018/M/LSZ1.pdf': ['LICENCSZERZŐ VERSENYSZÁM - LSZ1 FELADAT   \n \nEsemény:________________________________  Dátum: ___________  Bíró: ____________________  Pozíció:  \n \nIg. szám: ________  Lovas neve: _______________________  Startszám: ________  Ló neve: _______________________  \n \nIdőtartam  kb. 5  perc, 20x40 négyszög,  választható kantározás, összevont bíráskodás  \n  Feladat  Megjegyzés  \n1. A \nX \n \nC Belovaglás összeszedett ügetésben  \nÁllj, köszönés, elindulás összeszedett  \nügetésbe  \nJobb kézre   \n2. M-K \nK Átlóváltás, középügetés  \nÖsszeszedett  ügetés   \n3.  Átmenetek összeszedett ügetésből \nközépügetésbe és középügetésből \nösszeszedett ügetésbe   \n4. A \nD és X -H között  Középből  \nBalra oldaljárás   \n5. M-F Vállat be   \n6. A \nD és X -M között  Középből  \nJobbra oldaljárás   \n7. H-K \nK-F Vállat be  \nÖsszeszedett ügetés   \

In [72]:
for page_url in pdf_texts :
    page = pdf_texts[page_url]
    for program_url in page :
        program = page[program_url]
        print(program[0].split("\n")[0].strip())

LICENCSZERZŐ VERSENYSZÁM - LSZ1 FELADAT
LICENCSZERZŐ 7  FELADAT
LICENCSZERZŐ  5 FELADAT
DÍJLOVAGLÓ FELADAT M OSZTÁLY – LICENCSZERZŐ 6 FELADAT
LICENCSZERZŐ  4 FELADAT
LICENCSZERZŐ  2 FELADAT
DÍJLOVAGLÓ FELADAT M OSZTÁLY – LICENCSZERZŐ 5 FELADAT
LICENCSZERZŐ 4 FELADAT
DÍJLOVAGLÓ FELADAT M  OSZTÁLY – M ZENÉS KŰR
LICENCSZERZŐ  6 FELADAT
LICENCSZERZŐ  7 FELADAT
LICENCSZERZŐ  3 FELADAT
ELŐKÉSZÍTŐ PROGRAMOK – E1 FELADAT
DÍJLOVAGLÓ FELADAT E OSZTÁLY - E5 FELADA T
ELŐKÉSZÍTŐ E8 FELADAT – RAJTENGEDÉLY VIZSGA
DÍJLOVAGLÓ FELADAT E OSZTÁLY – E3 FELADA T
ELŐKÉSZÍTŐ PROGRAMOK - E5 FELADAT
ELŐKÉSZÍTŐ PROGRAMOK - E4 FELADAT
DÍJLOVAGLÓ FELADAT E OSZTÁLY – E8 FELADA T
ELŐKÉSZÍTŐ PROGRAMOK - E3 FELADAT

JUNIOR ZENÉS KŰR
JUNIOR EGYÉNI FELADAT
DÍJLOVAGLÓ FELADAT M OSZTÁLY – M5 FELADAT
DÍJLOVAGLÓ FELADAT M OSZTÁLY –M7FELADAT

DÍJLOVAGLÓ FELADAT M OSZTÁLY – M9 FELADAT
DÍJLOVAGLÓ FELADAT M OSZTÁLY –M6 FELADAT
JUNIOR CSAPATFELADAT
DÍJLOVAGLÓ FELADAT M OSZTÁLY –M4 FELADAT
DÍJLOVAGLÓ FELADAT M OSZTÁLY – M2 FELADA

In [90]:
pages = list(list(pdf_texts.values())[0].values())[0]

In [115]:
pages[0].split('\n')[0].strip()

'LICENCSZERZŐ VERSENYSZÁM - LSZ1 FELADAT'

In [116]:
text = pages[0] + pages[1]

In [117]:
import re

In [118]:
matches = re.findall("\n\d+\.", text)

In [122]:
program_points = []

remaining_text = text

last_point = 0

for match in matches :
    
    current_point = int(match[:-1])
    
    if current_point - 1 != last_point :
        break
    
    last_point = current_point
    
    split_text = remaining_text.split(match)
    if len(split_text) > 1 :
        remaining_text = split_text[1]
    program_points.append(remaining_text.split(split_text[0]))

In [124]:
len(program_points)

26